## Challenge code
 
 In this challenge you will be given the following functions: 
 - `create_Hamiltonian`: In which you build the Transverse Ising Hamiltonian for $N=4$ and a magnetic field intensity `h`. **You must complete this function.**
 - `model`: This QNode builds a general enough ansatz for the ground state. This circuit must depend on some parameters `params`, which you will later optimize. It returns the expectation value of the Hamiltonian for the output state of the circuit. **You must complete this function.**
 - `train`: This function returns the parameters that minimize the output of `model`. **You must complete this function.**
 
 ### Input
 
 As input to this problem, you are given:
 
 - `h` (`float`): Magnetic field intensity applied to the spin chain.
  
 ### Output
 
 This code will output a `float` corresponding to the energy of the ground state.
 
 If your solution matches the correct one within the given tolerance specified in `check` (in this case it's an relative tolerance of `0.1`), the output will be `"Correct!"` Otherwise, you will receive a `"Wrong answer"` prompt.
 
 Good luck!
 ### Imports
 The cell below specifies the libraries you should use in this challenge. Run the cell to import the libraries. ***Do not modify the cell.***

In [227]:
import json
import pennylane as qml
from pennylane import numpy as np

### Code
 Complete the code below. Note that during QHack, some sections were not editable. We've marked those sections accordingly here, but you can still edit them if you wish.

In [228]:
# Uneditable section #
def create_Hamiltonian(h):
    """
    Function in charge of generating the Hamiltonian of the statement.

    Args:
        h (float): magnetic field strength

    Returns:
        (qml.Hamiltonian): Hamiltonian of the statement associated to h
    """
    # End of uneditable section #

    # Put your code here #
    N = 4

    coeffs = []
    obs = []

    for i in range(N):
        next = (i+1) % N
        coeffs.append(-1)
        obs.append(qml.PauliZ(i) @ qml.PauliZ(next))

    for i in range(N):
        coeffs.append(-h)
        obs.append(qml.PauliX(i))

    return qml.Hamiltonian(coeffs, obs)


# Uneditable section #
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def model(params, H):
    """
    To implement VQE you need an ansatz for the candidate ground state!
    Define here the VQE ansatz in terms of some parameters (params) that
    create the candidate ground state. These parameters will
    be optimized later.

    Args:
        params (numpy.array): parameters to be used in the variational circuit
        H (qml.Hamiltonian): Hamiltonian used to calculate the expected value

    Returns:
        (float): Expected value with respect to the Hamiltonian H
    """

    # End of uneditable section #
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.RY(params[2], wires=2)
    qml.RY(params[3], wires=3)

    qml.RZ(params[4], wires=0)
    qml.RZ(params[5], wires=1)
    qml.RZ(params[6], wires=2)
    qml.RZ(params[7], wires=3)


    # Put your code here #
    return qml.expval(H)


# Uneditable section #
def train(h):
    """
    In this function you must design a subroutine that returns the
    parameters that best approximate the ground state.

    Args:
        h (float): magnetic field strength

    Returns:
        (numpy.array): parameters that best approximate the ground state.
    """

    # End of uneditable section #

    # Put your code here #
    H = create_Hamiltonian(h)

    opt = qml.GradientDescentOptimizer()
    params = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], requires_grad = True)
    max_iterations = 100

    energy_list = [model(params, H)]
    params_list = [params]

    def cost(params):
        return model(params, H)
    
    for n in range(max_iterations):
        params, prev_energy = opt.step_and_cost(cost, params)
        
        energy_list += [model(params, H)]
        params_list += [params]

        if n % 20 == 0:
            print("Iteration = {:},  Energy = {:.8f}".format(n, energy_list[-1]))

    return params

These functions are responsible for testing the solution. You will need to run the cell below. ***Do not modify the cell.***

In [229]:
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    params = train(ins)
    return str(model(params, create_Hamiltonian(ins)))


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-1
    ), "The expected value is not correct."

### Test cases
 Running the cell below will load the test cases. ***Do not modify the cell***.
 - input: 1.0
 	+ expected output: -5.226251859505506
 - input: 2.3
 	+ expected output: -9.66382463698038
 - input: 0.5
 	+ expected output: -4.271558410139714
 - input: 1.5
 	+ expected output: -6.760008550556145

In [230]:
test_cases = [['1.0', '-5.226251859505506'], ['2.3', '-9.66382463698038'], ['0.5', '-4.271558410139714'], ['1.5', '-6.760008550556145']]

### Solution testing
 Once you have run every cell above, including the one with your code, the cell below will test your solution. Run the cell. If you are correct for all of the test cases, it means your solutions is correct. Otherwise, you need to double check your work. ***Do not modify the cell below.***

In [231]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1.0'...
Iteration = 0,  Energy = -4.03959935
Iteration = 20,  Energy = -4.56805105
Iteration = 40,  Energy = -4.79913097
Iteration = 60,  Energy = -4.90306443
Iteration = 80,  Energy = -4.95174173
Correct!
Running test case 1 with input '2.3'...
Iteration = 0,  Energy = -4.20946572
Iteration = 20,  Energy = -6.91943502
Iteration = 40,  Energy = -8.00325122
Iteration = 60,  Energy = -8.48457687
Iteration = 80,  Energy = -8.72972510
Correct!
Running test case 2 with input '0.5'...
Iteration = 0,  Energy = -4.00989996
Iteration = 20,  Energy = -4.14274253
Iteration = 40,  Energy = -4.20167296
Iteration = 60,  Energy = -4.22801124
Iteration = 80,  Energy = -4.23991147
Correct!
Running test case 3 with input '1.5'...
Iteration = 0,  Energy = -4.08909669
Iteration = 20,  Energy = -5.26730611
Iteration = 40,  Energy = -5.76986349
Iteration = 60,  Energy = -5.99687676
Iteration = 80,  Energy = -6.10801028
Correct!
